In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import pymongo as pm
import pandas as pd
import numpy as np
from datetime import datetime
import urllib2

In [3]:
mongo_uri = "mongodb://ec2-52-38-154-245.us-west-2.compute.amazonaws.com:27017"
client = pm.MongoClient(mongo_uri)
auth = client.authenticate("dembois","coldlikeminnesota")
db = client['prod']

TypeError: 'Database' object is not callable. If you meant to call the 'authenticate' method on a 'MongoClient' object it is failing because no such method exists.

In [7]:
speakers = pd.read_csv('/Users/Rishi/Downloads/Election Polls - Sheet2.csv')

In [8]:
speakers

,name,speaker_id,party,state,incumbent,election_result,year
0,John F. Kennedy,kennedy_1,Democrat,Massachusetts,True,win,1960
1,Richard Nixon,nixon_1,Republican,California,False,lose,1960
2,Jimmy Carter,carter_1,Democrat,Georgia,False,win,1976
3,Gerald Ford,ford_1,Republican,Michigan,False,lose,1976
4,Jimmy Carter,carter_2,Democrat,Georgia,True,lose,1980
5,Ronald Reagan,reagan_1,Republican,California,False,win,1980
6,Walter Mondale,mondale_1,Democrat,Minnesota,False,lose,1984
7,Ronald Reagan,reagan_2,Republican,California,True,win,1984
8,Michael Dukakis,dukakis_1,Democrat,Massachusetts,False,lose,1988
9,George H.W. Bush,hwbush_1,Republican,Texas,False,win,1988


In [12]:
for index, row in speakers.iterrows():
    strdate = row['year']
    date = datetime.strptime(str(strdate), '%Y')
    speakers.loc[index, 'year'] = date

In [15]:
for index, row in speakers.iterrows():
    db.speakers.insert_one(
        {"name": row['name'],
         "speaker_id": row['speaker_id'],
         "party": row['party'],
         "state": row['state'],
        "incumbent": row['incumbent'],
        "election_result": row['election_result'],
        "year": row['year']}
    )

In [38]:
performance = pd.read_csv('/Users/Rishi/Downloads/Election Polls - Sheet1.csv')

In [39]:
performance

,Date of Debate,Debate ID,Pre-Debate Score Democrat,Pre-Debate Score Republican,Post-Debate Score Democrat,Post-Debate Score Republican,Democratic Candidate,Republican Candidate,Percent Change S&P
0,"September 26, 1960",0,47,47,46,47,kennedy_1,nixon_1,-0.226159
1,"October 7, 1960",1,46,47,49,46,kennedy_1,nixon_1,0.020369
2,"October 13, 1960",2,49,46,49,45,kennedy_1,nixon_1,0.531428
3,"September 23, 1976",3,51,40,47,45,carter_1,ford_1,-0.112233
4,"October 6, 1976",4,47,45,47,41,carter_1,ford_1,0.602410
5,"October 22, 1976",5,47,41,48,49,carter_1,ford_1,0.110044
6,"September 21, 1980",6,39,38,39,39,carter_2,reagan_1,0.889749
7,"October 28, 1980",7,45,39,44,47,carter_2,reagan_1,-0.109332
8,"October 7, 1984",8,38,58,39,56,mondale_1,reagan_2,-0.338087
9,"October 21, 1984",9,39,56,41,59,mondale_1,reagan_2,-0.357228


In [40]:
for index, row in performance.iterrows():
    strdate = row['Date of Debate']
    date = datetime.strptime(strdate, '%B %d, %Y')
    performance.loc[index, 'Date of Debate'] = date

In [43]:
for index, row in performance.iterrows():
    dspeaker = pd.Series(list(db.speakers.find({"speaker_id":row['Democratic Candidate']}))[0]).drop('_id')
    rspeaker =  pd.Series(list(db.speakers.find({"speaker_id":row['Republican Candidate']}))[0]).drop('_id')
    db.performance.insert_one(
        {"date": row['Date of Debate'],
         "debate_id": row['Debate ID'],
         "approval_rates":{
                "pre":{
                    "republican":row['Pre-Debate Score Republican'],
                    "democrat":row['Pre-Debate Score Democrat'],
                },
                "post":{
                    "republican":row['Post-Debate Score Republican'],
                    "democrat":row['Post-Debate Score Democrat']
                }
                
            },
         "percent_change_sp500":row['Percent Change S&P'],
         "speakers":{
                "republican":{"name":rspeaker['name'],
                     "speaker_id":rspeaker['speaker_id'],
                     "state": rspeaker['state'],
                     "incumbent": rspeaker['incumbent'],
                     "election_result": rspeaker['election_result'] if 'election_result' in rspeaker else '',
                     "year": rspeaker['year']
                    },
                "democrat":{"name":dspeaker['name'],
                     "speaker_id":dspeaker['speaker_id'],
                     "state": dspeaker['state'],
                     "incumbent": dspeaker['incumbent'],
                     "election_result": dspeaker['election_result'] if 'election_result' in dspeaker else '',
                     "year": dspeaker['year']
                    }
            }
         }
    )

KeyError: 'election_result'

In [72]:
debates = pd.read_csv('/Users/Rishi/Downloads/full_debates.csv')

In [75]:
debates['Date'] = pd.to_datetime(debates['Date'])

In [80]:
for indx, row in debates.iterrows():
    debates.loc[indx, 'Dem_Candidate'] = row['Dem_Candidate'].lower()
    debates.loc[indx, 'Rep_Candidate'] = row['Rep_Candidate'].lower()
    debates.loc[indx, 'indep_name'] = row['indep_name'].lower()

In [96]:
for indx, row in debates.iterrows():
    if row['Dem_Candidate'] != '\xe2\x80\x9c\xe2\x80\x9d':
        dspeaker = pd.Series(list(db.speakers.find({"speaker_id":row['Dem_Candidate']}))[0]).drop('_id')
    if row['Rep_Candidate'] != '\xe2\x80\x9c\xe2\x80\x9d':
        rspeaker =  pd.Series(list(db.speakers.find({"speaker_id":row['Rep_Candidate']}))[0]).drop('_id')
    if row['indep_name'] != '\xe2\x80\x9c\xe2\x80\x9d':
        ispeaker =  pd.Series(list(db.speakers.find({"speaker_id":row['indep_name']}))[0]).drop('_id')
    if 'ispeaker' in locals():
        db.debates.insert_one({
                "date": row['Date'],
                "speakers":{
                    "republican":{
                        "name":rspeaker['name'],
                        "speaker_id":rspeaker['speaker_id'],
                        "state": rspeaker['state'],
                        "incumbent": rspeaker['incumbent'],
                        "election_result": rspeaker['election_result'],
                        "year": rspeaker['year'],
                        "textblob": row['Rep_wordblob']
                    },
                    "independent":{
                        "name":ispeaker['name'],
                        "speaker_id":ispeaker['speaker_id'],
                        "state": ispeaker['state'],
                        "incumbent": ispeaker['incumbent'],
                        "election_result": ispeaker['election_result'],
                        "year": ispeaker['year'],
                        "textblob": row['Indep_word_blob']
                    },
                    "moderator":{
                        "textblob": row["Mod_wordblob"] if row["Mod_wordblob"] != '\xe2\x80\x9c\xe2\x80\x9d' else ""
                    }
                }
            })
        del ispeaker
    else:
        db.debates.insert_one({
                "date": row['Date'],
                "speakers":{
                    "republican":{
                        "name":rspeaker['name'],
                        "speaker_id":rspeaker['speaker_id'],
                        "state": rspeaker['state'],
                        "incumbent": rspeaker['incumbent'],
                        "election_result": rspeaker['election_result'],
                        "year": rspeaker['year'],
                        "textblob": row['Rep_wordblob']
                    },
                    "democrat":{
                        "name":dspeaker['name'],
                        "speaker_id":dspeaker['speaker_id'],
                        "state": dspeaker['state'],
                        "incumbent": dspeaker['incumbent'],
                        "election_result": dspeaker['election_result'],
                        "year": dspeaker['year'],
                        "textblob": row['Dem_Wordblob']
                    },
                    "moderator":{
                        "textblob": row["Mod_wordblob"] if row["Mod_wordblob"] != '\xe2\x80\x9c\xe2\x80\x9d' else ""
                    }
                }
            })

In [5]:
categories = pd.read_csv('/Users/Rishi/Downloads/inquirerbasic.csv')

/Users/Rishi/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (63,108,109,110,176) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [57]:
categories_dict = {}
for indx, row in categories.iterrows():
    currentry = ''
    for col, val in row[row.notnull()].iteritems():
        if col == "Entry":
            currentry = val.split('#')[0].lower()
        elif "Defined" != col and "Source" != col:
            if categories_dict.get(currentry, 'hi') != 'hi':
                x = categories_dict[currentry]
                x.append(val)
                categories_dict[currentry] = x
            else:
                categories_dict[currentry] = [val]


In [58]:
categories_dict

{'yellow': ['COLOR', 'Modif'],
 'four': ['Quan', 'NUMB', 'CARD', 'DET'],
 'centimeter': ['ABS', 'DIST', 'Space', 'TimeSpc', 'Noun'],
 'aggression': ['Negativ',
  'Ngtv',
  'Hostile',
  'Strong',
  'Active',
  'Vice',
  'PowCon',
  'PowTot',
  'Noun'],
 'conjure': ['Positiv', 'Strong', 'Active', 'Exprsv', 'Begin', 'IAV', 'SUPV'],
 'eligible': ['Virtue', 'PowOth', 'PowTot', 'Modif'],
 'electricity': ['NatrPro', 'WltOth', 'WltTot', 'Noun'],
 'rupture': ['Negativ',
  'Ngtv',
  'Vice',
  'Noun',
  'Negativ',
  'Ngtv',
  'Hostile',
  'Weak',
  'Exert',
  'DAV',
  'SUPV'],
 'rouse': ['Positiv', 'Active', 'SocRel', 'Exert', 'Causal', 'IAV', 'SUPV'],
 'scold': ['Negativ',
  'Ngtv',
  'Hostile',
  'Active',
  'ComForm',
  'DAV',
  'RspLoss',
  'RspTot',
  'SUPV PFREQ',
  'Negativ',
  'Ngtv',
  'Hostile',
  'Active',
  'ComForm',
  'COM',
  'RspOth',
  'RspTot',
  'FormLw',
  'Noun'],
 'originality': ['Positiv', 'Ovrst', 'Think', 'Quality', 'Noun'],
 'outwit': ['Positiv', 'Active', 'Think', 'Solv